# Comparing Models

This notebook chooses the most appropriate prompt and prompt structure for the OCR correction. 

In [1]:
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
from llm_comparison_toolkit import RateLimiter, get_response_openai, get_response_anthropic,  create_config_dict_func, compare_request_configurations, generate_model_configs
from evaluate import load
from evaluation_funcs import evaluate_correction_performance, evaluate_correction_performance_folders, get_metric_error_reduction
import seaborn as sns
import matplotlib.pyplot as plt
from helper_functions import files_to_df_func, files_to_df_core_func
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re


dev_data_folder = 'data/dev_data'
dev_transcripts = os.path.join(dev_data_folder, 'dev_data_transcript')
dev_raw_ocr_folder =  os.path.join(dev_data_folder,'dev_raw_ocr' )
dev_system_message_folder = os.path.join(dev_data_folder,'dev_system_message_variants' )


#NCSE

ncse_folder = 'data/transcription_returned_ocr'
ncse_articles_raw = os.path.join(ncse_folder, 'transcription_raw_ocr')
ncse_articles_transcribed = os.path.join(ncse_folder, 'transcription_files') 
ncse_articles_results = os.path.join(ncse_folder, 'corrected_folder')

#Overproof
overproof_folder = 'data/overproof'

smh_folder =  os.path.join(overproof_folder, 'SMH')
smh_articles_raw = os.path.join(smh_folder, 'article_level', 'raw')
smh_articles_transcribed = os.path.join(smh_folder, 'article_level', 'corrected') #in the dataset they are reffered to as corrected but this clashes with my naming convention
smh_articles_results = os.path.join(smh_folder, 'results')

ca_folder =  os.path.join(overproof_folder, 'CA')
ca_articles_raw = os.path.join(ca_folder, 'article_level', 'raw')
ca_articles_transcribed = os.path.join(ca_folder, 'article_level', 'corrected') #in the dataset they are reffered to as corrected but this clashes with my naming convention
ca_articles_results = os.path.join(ca_folder, 'results')

#load the dev and test sets for prompt development and selection
dev_data_df = pd.read_csv(os.path.join(dev_data_folder,'dev_data_raw.csv'))


#for saving data to be used in the analysis
if not os.path.exists('data/analysis'):
    os.makedirs('data/analysis')


wer = load("wer")
cer = load("cer")

## Evaluate system prompt tests

We evaluate the system prompts below to see if thre is any significant difference between the prompts


# Evaluating models on the test set

Having identified two different prompts and that the prompts appear to work better when places after the text we can now compare the different models


The below code creates the basic configuration dictionaries for each model and then fills in the with the two different prompt messages creating a single list of all basic prompt/model configurations. It then calls all the LLM's and saves the results.
This works in series so takes a while.

## Create API configurations

In [2]:
#Create the prompt/system message using the best performing from the previous section

full_prompt = "{content_html}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. The text is from an english newspaper in the 1800's. The text may be an advert or article and may be missing the beggining or end. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

instruct_prompt = "{content_html}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

boros_basic  = "{content_html}"+f""" \n \n """ +"Correct the text"

boros_complex  ="{content_html}"+f""" \n \n """ + f"""Please assist with reviewing and correcting errors in texts produced by automatic transcription (OCR) of historical documents.
Your task is to carefully examine the following text and correct any mistakes introduced by the OCR software. 
Do not write anything else than the corrected text."""

In [3]:
groq_alt_endpoint = {'alt_endpoint':{'base_url':'https://api.groq.com/openai/v1',
                     'api_key':os.getenv("GROQ_API_KEY")}}

basic_model_configs = pd.DataFrame({
    'get_response_func': [get_response_openai, get_response_openai, get_response_anthropic, get_response_anthropic, 
                          get_response_openai, get_response_openai, get_response_openai], 
    'engine': ['gpt-3.5-turbo', 'gpt-4-turbo-preview', "claude-3-haiku-20240307", "claude-3-opus-20240229", 
               'mixtral-8x7b-32768', 'llama2-70b-4096', 'gemma-7b-it'],
    'rate_limit':[160e3, 80e3, 100e3, 40e3, 9e3, 15e3, 15e3],
    'additional_args': [
        {}, {}, {}, {}, 
        groq_alt_endpoint, 
        groq_alt_endpoint, 
        groq_alt_endpoint
    ]
})

full_model_configs = generate_model_configs(basic_model_configs, full_prompt, 'full')
instruct_model_configs = generate_model_configs(basic_model_configs, instruct_prompt, 'instruct')

#I think on reflection I only need to compare boros complex on gpt-4 as this was the best performer in their paper
boros_configs = [
    (get_response_openai, 'gpt-4-turbo-preview', boros_complex, "boros_complex_"),
   # (get_response_openai, 'gpt-4-turbo-preview', boros_basic, "boros_basic_"),
  #  (get_response_anthropic, "claude-3-opus-20240229", boros_complex, "boros_complex_")
]

boros_list = [
    create_config_dict_func(
        get_response_func=config[0],
        rate_limiter=RateLimiter(80e3),
        engine=config[1],
        system_message_template="",
        prompt_template=config[2],
        additional_args={"response_name": config[3]}
    )
    for config in boros_configs
]

model_configs = full_model_configs + instruct_model_configs + boros_list



## Perform all API calls

The below section is what actually calls the API, the code points to the folders where the raw OCR is and provides a path to where the corrected text should be stored.

### NCSE

In [4]:
corrected_folder = ncse_articles_results
#
# This naming business needs to be cleaned up so the actual article ID is used. until then just have the following mess
#
# 

test_data_new = pd.read_csv(os.path.join(dev_data_folder,'transcription_raw_ocr.csv'))
test_data_new = test_data_new.loc[test_data_new ['file_name'].isin(files_to_df_func(ncse_articles_transcribed )['file_name'])] #subset to just the data I have transcribed

#This goes through the list of articles that have been transcribed, checks to see if there is a corrected version and if not generates it
compare_request_configurations(test_data_new, model_configs, folder_path=corrected_folder)

2024-04-22 21:59:51 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:00:03 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:00:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:00:22 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:00:29 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:00:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:00:59 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:01:36 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:01:47 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 

RateLimitError encountered: Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma-7b-it` in organization `org_01hqwmcenrfhzv88z47c50k8fc` on tokens per minute (TPM): Limit 7500, Used 6812, Requested ~2612. Please try again in 15.387s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}, waiting for a minute...


2024-04-22 22:42:07 httpx INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:42:08 httpx INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:42:10 httpx INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:42:10 httpx INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-04-22 22:42:10 openai._base_client INFO: Retrying request to /chat/completions in 2.000000 seconds
2024-04-22 22:42:13 httpx INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:42:21 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:42:24 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-22 22:42:28 httpx INFO: HTTP Request: POST https://api.openai.co

### Boros etal re-evaluation

The post-OCR correction worked so well that the Boros etal prompt is being re-evaluated

In [5]:
boros_complex  ="{content}"+f""" \n \n """ + f"""Please assist with reviewing and correcting errors in texts produced by automatic transcription (OCR) of historical documents.
Your task is to carefully examine the following text and correct any mistakes introduced by the OCR software. 
Do not write anything else than the corrected text."""


boros_config = generate_model_configs(basic_model_configs.iloc[0:2, :], boros_complex, 'boros_complex_')[1]

### Sydney Morning Herald

This section performs the correction test on dataset 2 of the Overproof collection. This is data from the Sydney Morning Herald. In addition it re-tests the Boros et al.

In [6]:
smh_data = files_to_df_core_func(smh_articles_raw )

smh_data['content'] = smh_data['content'].str.replace('\n', ' ')

smh_data['id'] = smh_data['file_name'] # this is needed as the processing log uses an id to keep track of what has been processed and what hasn't to allow for easy restarts

full_prompt_smh = "{content}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. The text is from The Sydney Morning Herald 1842 -1950. The text may be an advert or article and may be missing the beggining or end. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

instruct_prompt_smh = "{content}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

full_model_configs_smh = generate_model_configs(basic_model_configs, full_prompt_smh, 'full')
instruct_model_configs_smh = generate_model_configs(basic_model_configs, instruct_prompt_smh, 'instruct')

#Boros et al prompt added in as the overall system works so well, it seems strange theirs didn't work, this is a quick check
smh_configs = full_model_configs_smh + instruct_model_configs_smh  + [boros_config ]

corrected_folder_smh = smh_articles_results

compare_request_configurations(smh_data, smh_configs, folder_path=corrected_folder_smh)

### Chronicalling America

This section performs the correction test on dataset 2 of the Overproof collection. This is data from the Chronicalling America Dataset. 

In [7]:
ca_data = files_to_df_core_func(ca_articles_raw )

ca_data['content'] = ca_data['content'].str.replace('\n', ' ')

ca_data['id'] = ca_data['file_name'] # this is needed as the processing log uses an id to keep track of what has been processed and what hasn't to allow for easy restarts

full_prompt_ca = "{content}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. The text is from American Newspapers 1870 -1922. The text may be an advert or article and may be missing the beggining or end. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""

instruct_prompt_ca = "{content}"+f""" \n \n """ + f"""You are an expert in post-OCR correction of documents. Using the context available from the text please recover the most likely original text from the corrupted OCR. Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."""


full_model_configs_ca = generate_model_configs(basic_model_configs, full_prompt_ca, 'full')
instruct_model_configs_ca = generate_model_configs(basic_model_configs, instruct_prompt_ca, 'instruct')

ca_configs = full_model_configs_ca + instruct_model_configs_ca  + [boros_config ]

corrected_folder_ca = ca_articles_results

compare_request_configurations(ca_data, ca_configs, folder_path=corrected_folder_ca)

## Evaluate the prompts across all models

On the smaller models, Full is worse than instruct on the larger models the reverse. Maybe this is related to ability to 'focus' or hold isntructions in memory?

In [12]:
##
## double check get_metric_error_reduction
##

corrected_folder = ncse_articles_results 

gt_folder = ncse_articles_transcribed 

raw_ocr = ncse_articles_raw

ncse_performance_eval =  evaluate_correction_performance_folders(corrected_folder, gt_folder, wer, cer)

ncse_raw_ocr_eval =  evaluate_correction_performance(raw_ocr, gt_folder , wer, cer, 'raw_ocr')


ncse_error_reduction = get_metric_error_reduction(ncse_performance_eval, ncse_raw_ocr_eval )

ncse_error_reduction.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER            CER         lev_dist  \
                                      mean    50%    mean     50%     mean   
type                                                                         
full__gemma-7b-it                   -15.54 -16.89 -518.93 -438.49  -505.81   
instruct__claude-3-haiku-20240307     7.66   4.38 -451.43 -416.94  -440.49   
full__claude-3-haiku-20240307        -1.91   9.95 -540.02 -400.21  -527.16   
instruct__gemma-7b-it                -8.95   3.95 -527.45 -366.23  -514.25   
instruct__mixtral-8x7b-32768         25.01  39.03 -180.88 -160.83  -174.98   
full__mixtral-8x7b-32768             25.15  26.37 -159.35 -157.87  -154.51   
instruct__llama2-70b-4096            39.17  49.56  -63.93  -60.52   -63.22   
full__gpt-3.5-turbo                  21.83  38.78 -267.23  -54.36  -259.58   
full__llama2-70b-4096              -208.80  58.62 -667.16  -28.81  -653.78   
boros_basic__gpt-4-turbo-preview     83.78  86.07   71.34   70.93    67.55   
instruct__gpt-4-turbo-preview        84.36  85.97   74.95   76.62    71.46   
full__gpt-4-turbo-preview            82.29  82.31   72.06   77.33    68.58   
instruct__gpt-3.5-turbo              39.50  82.81  -35.65   80.65   -36.22   
boros_complex__gpt-4-turbo-preview   82.49  83.42   73.19   81.26    69.75   
full__claude-3-opus-20240229         81.98  83.68   73.73   85.42    69.79   
instruct__claude-3-opus-20240229     83.66  84.74   76.27   86.01    72.21   

                                            
                                       50%  
type                                        
full__gemma-7b-it                  -425.19  
instruct__claude-3-haiku-20240307  -404.33  
full__claude-3-haiku-20240307      -388.23  
instruct__gemma-7b-it              -355.19  
instruct__mixtral-8x7b-32768       -157.34  
full__mixtral-8x7b-32768           -151.92  
instruct__llama2-70b-4096           -59.65  
full__gpt-3.5-turbo                 -53.77  
full__llama2-70b-4096               -29.68  
boros_basic__gpt-4-turbo-preview     66.11  
instruct__gpt-4-turbo-preview        72.75  
full__gpt-4-turbo-preview            73.43  
instruct__gpt-3.5-turbo              76.66  
boros_complex__gpt-4-turbo-preview   76.76  
full__claude-3-opus-20240229         80.09  
instruct__claude-3-opus-20240229     80.65

In [9]:
corrected_folder = smh_articles_results 

gt_folder = smh_articles_transcribed 

raw_ocr = smh_articles_raw

smh_performance_eval =  evaluate_correction_performance_folders(corrected_folder, gt_folder, wer, cer)

smh_raw_ocr_eval =  evaluate_correction_performance(raw_ocr, gt_folder , wer, cer, 'raw_ocr')

smh_error_reduction = get_metric_error_reduction(smh_performance_eval, smh_raw_ocr_eval )

smh_error_reduction.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER            CER        lev_dist  \
                                      mean    50%    mean    50%     mean   
type                                                                        
full__gemma-7b-it                    -9.43   3.26 -227.08 -35.65  -225.59   
instruct__mixtral-8x7b-32768         13.60  17.39  -73.26 -19.11   -73.68   
full__mixtral-8x7b-32768             13.40  18.00  -58.17 -14.63   -59.16   
instruct__gemma-7b-it                 2.66   9.66 -161.36 -12.93  -160.68   
full__llama2-70b-4096                 0.68  16.67  -43.20   5.77   -45.81   
instruct__llama2-70b-4096            -2.28  16.00  -44.03   6.45   -46.28   
overproof                            19.38  19.23   27.28  28.38    25.83   
instruct__claude-3-haiku-20240307    21.35  26.92  -48.01  35.71   -47.86   
full__claude-3-haiku-20240307        21.58  27.34  -40.65  38.38   -40.69   
full__gpt-3.5-turbo                  25.34  26.72   -0.52  39.18    -0.34   
instruct__gpt-4-turbo-preview        29.88  30.20   32.09  41.79    31.60   
full__gpt-4-turbo-preview            30.15  30.00   32.65  42.08    31.98   
instruct__gpt-3.5-turbo              27.18  27.75   28.45  42.86    28.13   
instruct__claude-3-opus-20240229     30.96  32.72   36.15  45.45    32.12   
boros_complex___gpt-4-turbo-preview  30.20  30.71   44.01  48.44    43.46   
full__claude-3-opus-20240229         31.39  32.11   45.92  51.03    41.79   

                                            
                                       50%  
type                                        
full__gemma-7b-it                   -35.34  
instruct__mixtral-8x7b-32768        -20.00  
full__mixtral-8x7b-32768            -15.24  
instruct__gemma-7b-it               -13.04  
full__llama2-70b-4096                 1.85  
instruct__llama2-70b-4096             3.08  
overproof                            27.59  
instruct__claude-3-haiku-20240307    33.80  
full__claude-3-haiku-20240307        37.48  
full__gpt-3.5-turbo                  38.66  
instruct__gpt-4-turbo-preview        41.18  
full__gpt-4-turbo-preview            41.46  
instruct__gpt-3.5-turbo              41.46  
instruct__claude-3-opus-20240229     41.57  
boros_complex___gpt-4-turbo-preview  48.23  
full__claude-3-opus-20240229         48.78

In [10]:
corrected_folder = ca_articles_results 

gt_folder = ca_articles_transcribed 

raw_ocr = ca_articles_raw

ca_performance_eval =  evaluate_correction_performance_folders(corrected_folder, gt_folder, wer, cer)

ca_raw_ocr_eval =  evaluate_correction_performance(raw_ocr, gt_folder , wer, cer, 'raw_ocr')

ca_error_reduction = get_metric_error_reduction(ca_performance_eval, ca_raw_ocr_eval )

ca_error_reduction.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER            CER        lev_dist  \
                                      mean    50%    mean    50%     mean   
type                                                                        
instruct__gemma-7b-it                -0.83   5.66 -154.39 -41.00  -153.98   
full__gemma-7b-it                    -5.28   3.04 -209.20 -38.01  -208.30   
instruct__mixtral-8x7b-32768          7.89  13.72  -71.77 -22.07   -72.00   
full__mixtral-8x7b-32768              8.66  15.72  -60.65 -16.30   -60.94   
full__llama2-70b-4096                10.00  10.19  -35.66  -4.90   -38.95   
instruct__llama2-70b-4096             7.72  10.86  -45.09  -6.48   -46.74   
full__claude-3-haiku-20240307         8.15  14.97  -92.14  26.01   -92.22   
overproof                            20.64  21.63   34.11  34.59    34.01   
instruct__claude-3-haiku-20240307     9.48  17.41  -91.40  34.73   -91.28   
instruct__gpt-4-turbo-preview        22.64  20.98   33.74  37.57    33.23   
full__gpt-4-turbo-preview            22.12  21.84   29.70  38.18    28.86   
instruct__claude-3-opus-20240229     24.90  24.26   41.26  47.01    38.70   
full__claude-3-opus-20240229         24.74  24.61   43.51  48.23    40.76   
full__gpt-3.5-turbo                  19.48  22.90  -18.29  44.22   -18.35   
instruct__gpt-3.5-turbo              25.64  24.62   41.21  44.05    40.96   
boros_complex___gpt-4-turbo-preview  24.61  24.02   44.23  45.55    43.95   

                                            
                                       50%  
type                                        
instruct__gemma-7b-it               -42.38  
full__gemma-7b-it                   -38.15  
instruct__mixtral-8x7b-32768        -22.36  
full__mixtral-8x7b-32768            -14.86  
full__llama2-70b-4096                -8.14  
instruct__llama2-70b-4096            -7.94  
full__claude-3-haiku-20240307        25.27  
overproof                            34.59  
instruct__claude-3-haiku-20240307    34.73  
instruct__gpt-4-turbo-preview        37.45  
full__gpt-4-turbo-preview            37.70  
instruct__claude-3-opus-20240229     41.18  
full__claude-3-opus-20240229         43.44  
full__gpt-3.5-turbo                  43.99  
instruct__gpt-3.5-turbo              44.05  
boros_complex___gpt-4-turbo-preview  45.19

In [13]:
ncse_median = ncse_error_reduction.groupby('type')['lev_dist'].median().round(2)
smh_median = smh_error_reduction.groupby('type')['lev_dist'].median().round(2)
ca_median = ca_error_reduction.groupby('type')['lev_dist'].median().round(2)

# Combine the results into a new dataframe
result_df = pd.DataFrame({
    'NCSE': ncse_median,
    'SMH': smh_median,
    'CA': ca_median
})

# Reset the index to make 'type' a regular column
result_df = result_df.reset_index()

result_df = result_df.loc[~result_df['type'].isin(['boros_basic__gpt-4-turbo-preview',  'claude_temp_claude-3-opus-20240229'])]

result_df.sort_values('NCSE')

,type,NCSE,SMH,CA
5,full__gemma-7b-it,-425.19,-35.34,-38.15
10,instruct__claude-3-haiku-20240307,-404.33,33.80,34.73
3,full__claude-3-haiku-20240307,-388.23,37.48,25.27
12,instruct__gemma-7b-it,-355.19,-13.04,-42.38
16,instruct__mixtral-8x7b-32768,-157.34,-20.00,-22.36
9,full__mixtral-8x7b-32768,-151.92,-15.24,-14.86
15,instruct__llama2-70b-4096,-59.65,3.08,-7.94
6,full__gpt-3.5-turbo,-53.77,38.66,43.99
8,full__llama2-70b-4096,-29.68,1.85,-8.14
14,instruct__gpt-4-turbo-preview,72.75,41.18,37.45
